### Purpose

The purpose of this file is to get the distances between fires and my towns, saving only those who have distances less that 1250 miles.

Changes feature geometry into different coordinate system

In [30]:
#
#    IMPORTS
# 

#    Import some standard python modules
import os, json, time
#
#    The module pyproj is a standard module that can be installed using pip or your other favorite
#    installation tool. This module provides tools to convert between different geodesic coordinate systems
#    and for calculating distances between points (coordinates) in a specific geodesic system.
#
from pyproj import Transformer, Geod
#
#    The 'wildfire' module is a user module. This module is available from the course website. The module
#    includes one object, a Reader, that can be used to read the GeoJSON files associated with the
#    wildefire dataset. The module also contains a sample datafile that is GeoJSON compliant and that
#    contains a small number of California wildfires extracted from the main wildfire dataset.
#    
from wildfire.Reader import Reader as WFReader
#
#    There is a GeoJSON reader that you might try if you wanted to read the data. It has its own quirks.
#    There will be an example below that reads the sample file "Wildfire_short_sample.json"
#    
import geojson
#
from datetime import datetime

import pandas as pd

In [6]:
#
#    Transform feature geometry data
#
#    The function takes one parameter, a list of ESRI:102008 coordinates that will be transformed to EPSG:4326
#    The function returns a list of coordinates in EPSG:4326
def convert_ring_to_epsg4326(ring_data=None):
    converted_ring = list()
    #
    # We use a pyproj transformer that converts from ESRI:102008 to EPSG:4326 to transform the list of coordinates
    to_epsg4326 = Transformer.from_crs("ESRI:102008","EPSG:4326")
    # We'll run through the list transforming each ESRI:102008 x,y coordinate into a decimal degree lat,lon
    for coord in ring_data:
        lat,lon = to_epsg4326.transform(coord[0],coord[1])
        new_coord = lat,lon
        converted_ring.append(new_coord)
    return converted_ring

Finds the shortest distance between a place and a fire

In [7]:
#    
#    The function takes two parameters
#        A place - which is coordinate point (list or tuple with two items, (lat,lon) in decimal degrees EPSG:4326
#        Ring_data - a list of decimal degree coordinates for the fire boundary
#
#    The function returns a list containing the shortest distance to the perimeter and the point where that is
#
def shortest_distance_from_place_to_fire_perimeter(place=None,ring_data=None):
    # convert the ring data to the right coordinate system
    ring = convert_ring_to_epsg4326(ring_data)    
    # create a epsg4326 compliant object - which is what the WGS84 ellipsoid is
    geodcalc = Geod(ellps='WGS84')
    closest_point = list()
    # run through each point in the converted ring data
    for point in ring:
        # calculate the distance
        d = geodcalc.inv(place[1],place[0],point[1],point[0])
        # convert the distance to miles
        distance_in_miles = d[2]*0.00062137
        # if it's closer to the city than the point we have, save it
        if not closest_point:
            closest_point.append(distance_in_miles)
            closest_point.append(point)
        elif closest_point and closest_point[0]>distance_in_miles:
            closest_point = list()
            closest_point.append(distance_in_miles)
            closest_point.append(point)
    return closest_point


Reads the wildfire data

In [10]:
#
#    Open a file, load it with the geojson loader
#
#EMILY ADD BELOW
SAMPLE_DATA_FILENAME = '../intermediate_data/filtered_fires.json'
print(f"Attempting to open '{SAMPLE_DATA_FILENAME}'")
geojson_file = open(SAMPLE_DATA_FILENAME,"r")
print(f"Using GeoJSON module to load sample file '{SAMPLE_DATA_FILENAME}'")
gj_data = geojson.load(geojson_file)
geojson_file.close()
#
#    Print the keys from the object
#
'''gj_keys = list(gj_data.keys())
print("The loaded JSON dictionary has the following keys:")
print(gj_keys)
print()
#
#    For all GeoJSON type things, the most important part of the file are the 'features'. 
#    In the case of the wildfire dataset, each feature is a polygon (ring) of points that define the bounary of a fire
#
count = 0
for feature in gj_data['features']:
    count += 1
    #print(json.dumps(feature,indent=4))
    time.sleep(0.5)   # this slows the output to fix output rate limits for Jupyter

print(f"Found {count} features in the variable 'gj_data' ")
#'''

Attempting to open '../intermediate_data/filtered_fires.json'
Using GeoJSON module to load sample file '../intermediate_data/filtered_fires.json'
The loaded JSON dictionary has the following keys:
['displayFieldName', 'fieldAliases', 'geometryType', 'spatialReference', 'fields', 'features']



KeyboardInterrupt: 

In [37]:
#
#    This bit of code opens a new wildfire reader, gets the header information and prints it to the screen
#
SAMPLE_DATA_FILENAME = '../../USGS_Wildland_Fire_Combined_Dataset.json'
print(f"Attempting to open '{SAMPLE_DATA_FILENAME}' with wildfire.Reader() object")
wfreader = WFReader(SAMPLE_DATA_FILENAME)
print()
#
#    Now print the header - it contains some useful information
#
header_dict = wfreader.header()
header_keys = list(header_dict.keys())
print("The header has the following keys:")
print(gj_keys)
print()
print("Header Dictionary")
print(json.dumps(header_dict,indent=4))


Attempting to open '../../USGS_Wildland_Fire_Combined_Dataset.json' with wildfire.Reader() object

The header has the following keys:
['displayFieldName', 'fieldAliases', 'geometryType', 'spatialReference', 'fields', 'features']

Header Dictionary
{
    "displayFieldName": "",
    "fieldAliases": {
        "OBJECTID": "OBJECTID",
        "USGS_Assigned_ID": "USGS Assigned ID",
        "Assigned_Fire_Type": "Assigned Fire Type",
        "Fire_Year": "Fire Year",
        "Fire_Polygon_Tier": "Fire Polygon Tier",
        "Fire_Attribute_Tiers": "Fire Attribute Tiers",
        "GIS_Acres": "GIS_Acres",
        "GIS_Hectares": "GIS_Hectares",
        "Source_Datasets": "Source Datasets",
        "Listed_Fire_Types": "Listed Fire Types",
        "Listed_Fire_Names": "Listed Fire Names",
        "Listed_Fire_Codes": "Listed Fire Codes",
        "Listed_Fire_IDs": "Listed Fire IDs",
        "Listed_Fire_IRWIN_IDs": "Listed Fire IRWIN IDs",
        "Listed_Fire_Dates": "Listed Fire Dates",
    

Getting a list of features (fires) in the data

In [40]:
#
#    This sample code will load the whole sample file, or a small amount of the complete dataset.
#
MAX_FEATURE_LOAD = 100
feature_list = list()
feature_count = 0
# A rewind() on the reader object makes sure we're at the start of the feature list
# This way, we can execute this cell multiple times and get the same result 
wfreader.rewind()
# Now, read through each of the features, saving them as dictionaries into a list
feature = wfreader.next()
while feature:
    feature_list.append(feature)
    feature_count += 1
    # if we're loading a lot of features, print progress
    if (feature_count % 1000) == 0:
        print(f"Loaded {feature_count} features")
    # loaded the max we're allowed then break
    '''if feature_count >= MAX_FEATURE_LOAD:
        break'''
    feature = wfreader.next()
#
#    Print the number of items (features) we think we loaded
print(f"Loaded a total of {feature_count} features")
#
#    Just a validation check - did all the items we loaded get into the list?
print(f"Variable 'feature_list' contains {len(feature_list)} features")

Loaded 1000 features
Loaded 2000 features
Loaded 3000 features
Loaded 4000 features
Loaded 5000 features
Loaded 6000 features
Loaded 7000 features
Loaded 8000 features
Loaded 9000 features
Loaded 10000 features
Loaded 11000 features
Loaded 12000 features
Loaded 13000 features
Loaded 14000 features
Loaded 15000 features
Loaded 16000 features
Loaded 17000 features
Loaded 18000 features
Loaded 19000 features
Loaded 20000 features
Loaded 21000 features
Loaded 22000 features
Loaded 23000 features
Loaded 24000 features
Loaded 25000 features
Loaded 26000 features
Loaded 27000 features
Loaded 28000 features
Loaded 29000 features
Loaded 30000 features
Loaded 31000 features
Loaded 32000 features
Loaded 33000 features
Loaded 34000 features
Loaded 35000 features
Loaded 36000 features
Loaded 37000 features
Loaded 38000 features
Loaded 39000 features
Loaded 40000 features
Loaded 41000 features
Loaded 42000 features
Loaded 43000 features
Loaded 44000 features
Loaded 45000 features
Loaded 46000 featur

Actually calling the distance methods on our city below

In [41]:
#Building out the city location
CITY_LOCATIONS = {
    'pahrump' :     {'city'   : 'Pahrump', 
                     'latlon' : [36.231143, -116.017339]}}

In [47]:
#    Get a city from our CITY_LOCATIONS constant as our starting position
place = CITY_LOCATIONS["pahrump"]

fire_id = []
shortest_dist_from_edge = []
features_processed = 0

for wf_feature in feature_list:
    #Try/Except to catch fires which aren't in a ring shape
    try:
        ring_data = wf_feature['geometry']['rings'][0]
        distance = shortest_distance_from_place_to_fire_perimeter(place['latlon'],ring_data)
        fire_id.append(wf_feature['attributes']['OBJECTID'])
        shortest_dist_from_edge.append(round(distance[0], 2))
    except KeyError:
        print("{0} fire is in {1} shape, ignoring.".format(wf_feature['attributes']['OBJECTID'], wf_feature['geometry']))
    #Incrementing the fires processed counter and saving every 10000 entries to avoid lost work
    features_processed = features_processed + 1
    if features_processed % 1000 == 0:
        print("Processed {0} features".format(features_processed))
    if features_processed % 10000 == 0:
        dist_df = pd.DataFrame({'OBJECTID': fire_id, 'shortest_dist': shortest_dist_from_edge})
        dist_df.to_csv('../intermediate_data/fire_distances.csv', index=False)

#Saving the final file
dist_df = pd.DataFrame({'OBJECTID': fire_id, 'shortest_dist': shortest_dist_from_edge})
dist_df.to_csv('../intermediate_data/fire_distances.csv', index=False)

Processed 1000 features
Processed 2000 features
Processed 3000 features
Processed 4000 features
Processed 5000 features
Processed 6000 features
Processed 7000 features
Processed 8000 features
Processed 9000 features
Processed 10000 features
Processed 11000 features
Processed 12000 features
Processed 13000 features
Processed 14000 features
Processed 15000 features
Processed 16000 features
Processed 17000 features
Processed 18000 features
Processed 19000 features
Processed 20000 features
Processed 21000 features
Processed 22000 features
Processed 23000 features
Processed 24000 features
Processed 25000 features
Processed 26000 features
Processed 27000 features
Processed 28000 features
Processed 29000 features
Processed 30000 features
Processed 31000 features
Processed 32000 features
Processed 33000 features
Processed 34000 features
Processed 35000 features
Processed 36000 features
Processed 37000 features
Processed 38000 features
Processed 39000 features
Processed 40000 features
Processed

ValueError: cannot switch from manual field specification to automatic field numbering

In [ ]:
#REMOVE AFTER RUNNING
#FIXING THE REST OF THE FIRES
place = CITY_LOCATIONS["pahrump"]

fire_id = []
shortest_dist_from_edge = []
features_processed = 0

for wf_feature in feature_list:
    if wf_feature['attributes']['OBJECTID'] > 100000:
    #Try/Except to catch fires which aren't in a ring shape
        try:
            ring_data = wf_feature['geometry']['rings'][0]
            distance = shortest_distance_from_place_to_fire_perimeter(place['latlon'],ring_data)
            fire_id.append(wf_feature['attributes']['OBJECTID'])
            shortest_dist_from_edge.append(round(distance[0], 2))
        except KeyError:
            print("{0} fire is in {1} shape, ignoring.".format(wf_feature['attributes']['OBJECTID'], list(wf_feature['geometry'].keys())[0]))
        #Incrementing the fires processed counter and saving every 10000 entries to avoid lost work
    features_processed = features_processed + 1
    if features_processed % 1000 == 0:
        print("Processed {0} features".format(features_processed))
    if features_processed % 10000 == 0:
        dist_df = pd.DataFrame({'OBJECTID': fire_id, 'shortest_dist': shortest_dist_from_edge})
        dist_df.to_csv('../intermediate_data/fire_distances_rest.csv', index=False)
    

dist_df = pd.DataFrame({'OBJECTID': fire_id, 'shortest_dist': shortest_dist_from_edge})
dist_df.to_csv('../intermediate_data/fire_distances_rest.csv', index=False)

Processed 1000 features
Processed 2000 features
Processed 3000 features
Processed 4000 features
Processed 5000 features
Processed 6000 features
Processed 7000 features
Processed 8000 features
Processed 9000 features
Processed 10000 features
Processed 11000 features
Processed 12000 features
Processed 13000 features
Processed 14000 features
Processed 15000 features
Processed 16000 features
Processed 17000 features
Processed 18000 features
Processed 19000 features
Processed 20000 features
Processed 21000 features
Processed 22000 features
Processed 23000 features
Processed 24000 features
Processed 25000 features
Processed 26000 features
Processed 27000 features
Processed 28000 features
Processed 29000 features
Processed 30000 features
Processed 31000 features
Processed 32000 features
Processed 33000 features
Processed 34000 features
Processed 35000 features
Processed 36000 features
Processed 37000 features
Processed 38000 features
Processed 39000 features
Processed 40000 features
Processed

Processed 110000 features
110224 fire is in {'curveRings': [[[974392.1541999988, -534800.9936999995], [974390.4675999992, -534732.1742000002], [974403.3416000009, -534704.0697000008], [974414.6900999993, -534688.5835999995], [974434.8211999983, -534677.6160000004], [974455.2993000001, -534663.3527000006], [974477.2958999984, -534658.2142999992], [974505.1182999983, -534658.2288000006], [974526.5725000016, -534673.1030000001], [974527.1075000018, -534686.5318], [974511.116799999, -534716.5197000001], [974512.261500001, -534737.8851999994], [974524.2611000016, -534761.1184], [974534.8046000004, -534774.6939000003], [974573.5654999986, -534796.5287999995], [974623.5320000015, -534816.9541999996], [974648.0505000018, -534820.8990000002], [974662.8859999999, -534823.2502999995], [974678.6405000016, -534817.7539000008], [974688.5714999996, -534801.0402000006], [974704.2397999987, -534794.8516000006], [974723.5381999984, -534792.4239000008], [974738.9101000018, -534794.0047999993], [974758.62

Processed 111000 features
111431 fire is in {'curveRings': [[[1124302.3839999996, -656249.7921999991], {'a': [[1124268.9450999983, -655927.6524999999], [1124342.150340564, -656082.1367563709], 1, 1, 0.13739895298126142, 212.07735369676428, 0.7924321696529887]}, {'a': [[1124295.5423000008, -655919.1168000009], [1124328.3603315717, -656043.3175052362], 1, 1, 0.1695329148825172, 152.3743322976385, 0.8421127946491282]}, {'a': [[1124307.1372000016, -655917.1592999995], [1124323.8810628331, -656043.5848083502], 1, 1, 0.1350624553784235, 146.96070061075162, 0.8677781896933973]}, {'a': [[1124409.2762999982, -655943.9849999994], [1124311.6805786125, -656048.767883082], 1, 1, 0.037888633364385374, 158.1943506433491, 0.8324365010684242]}, {'a': [[1124442.0373000018, -655975.1554000005], [1124293.5452189457, -656056.3986326605], 1, 1, 6.271033537988047, 177.85664141484443, 0.8355059926334539]}, {'a': [[1124463.4538000003, -656009.4305000007], [1124285.1345005997, -656066.1320932871], 1, 1, 0.01268

111897 fire is in {'curveRings': [[[-1936451.6747999992, -107721.65520000085], [-1936439.3997000009, -107759.43969999999], [-1936399.5346000008, -107744.93050000072], [-1936366.2886999995, -107714.48440000042], [-1936364.9197000004, -107710.59769999981], [-1936363.8849, -107707.09380000085], [-1936363.1460999995, -107703.84200000018], [-1936362.6958000008, -107700.80639999919], [-1936362.3561000004, -107696.39589999989], [-1936362.3490999993, -107692.73039999977], [-1936362.6831, -107689.51149999909], [-1936363.3683000002, -107686.7770000007], [-1936363.5294000003, -107686.41860000044], [-1936364.4575999994, -107684.35449999943], [-1936366.0856999997, -107681.85419999994], [-1936368.2537999991, -107679.2455000002], [-1936371.4359000009, -107675.95869999938], [-1936373.3085999992, -107674.35490000062], [-1936375.6209999993, -107672.85610000044], [-1936381.1932999995, -107670.10209999979], {'b': [[-1936384.4649, -107668.48520000093], [-1936382.300626505, -107669.58567463467], [-1936383.4

Processed 112000 features
112410 fire is in {'curveRings': [[[948556.4836000018, -604134.5874000005], [948555.8794, -604129.3267000001], [948554.2199000008, -604108.2864999995], [948553.3121999986, -604087.2098999992], [948553.1574000008, -604066.1295999996], [948553.756000001, -604045.0788000003], [948555.1068999991, -604024.0899999999], [948557.2080000006, -604003.1962000001], [948560.0560999997, -603982.4299999997], [948563.6466999985, -603961.8237999994], [948567.9743000008, -603941.4097000007], [948573.0320000015, -603921.2195999995], [948578.8117999993, -603901.2852999996], [948585.3049000017, -603881.6375999991], [948592.5012000017, -603862.3071999997], [948600.3891999982, -603843.3244000003], [948608.9567999989, -603824.7188000008], [948618.1906000003, -603806.5196000002], [948628.0760999992, -603788.7548999991], [948638.5978999995, -603771.4526000004], [948649.739599999, -603754.6399000008], [948661.4836999997, -603738.3425999992], [948673.811999999, -603722.5866999999], [9486

...

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Processed 124000 features
124535 fire is in {'curveRings': [[[-1984521.7230999991, 162846.29030000046], [-1984521.519200001, 162846.20199999958], [-1984521.545499999, 162845.9263000004], [-1984521.7230999991, 162846.29030000046]], [[-1984521.7230999991, 162846.29030000046], [-1984546.2906999998, 162856.86500000022], [-1984590.3609999996, 162897.7628000006], [-1984679.6099999994, 162910.28219999932], [-1984714.3922000006, 162913.24400000088], [-1984734.7493999992, 162921.81389999948], [-1984743.3164000008, 162924.51799999923], [-1984748.8692000005, 162929.50320000015], [-1984757.7442000005, 162931.9746000003], [-1984759.0050000008, 162931.34659999982], [-1984766.2467999998, 162929.21279999986], [-1984768.7162999995, 162926.90139999986], [-1984769.8950999994, 162926.56909999996], [-1984771.012599999, 162925.3546999991], [-1984772.2655999996, 162925.3973999992], [-1984773.2080000006, 162924.22509999946], [-1984779.5395, 162919.83070000075], [-1984792.1368000004, 162915.52800000086], [-198

Processed 125000 features
125046 fire is in {'curveRings': [[[-1950365.8090000004, -63693.51099999994], [-1950384.2924000006, -63713.37529999949], [-1950406.6852000002, -63704.941700000316], [-1950409.9772999994, -63666.69810000062], [-1950410.2576000001, -63663.44209999964], [-1950391.2112000007, -63648.069199999794], [-1950369.0417, -63653.77679999918], [-1950361.2272999994, -63657.004100000486], [-1950352.740700001, -63674.68480000086], [-1950365.8090000004, -63693.51099999994]], [[-1950428.7499000002, -62881.944399999455], [-1950418.2526999991, -62881.88959999941], [-1950407.7423999999, -62885.19099999964], [-1950402.1699, -62887.561599999666], [-1950400.6919999998, -62889.5214000009], [-1950399.1204000004, -62892.56000000052], [-1950398.5949000008, -62895.382099999115], [-1950397.9264000002, -62925.2145000007], [-1950397.7420000006, -62928.101999999955], [-1950397.3205999993, -62930.94490000047], [-1950396.6653000005, -62933.71959999949], [-1950395.7817000002, -62936.403200000525]

125745 fire is in {'curveRings': [[[-100345.76970000006, 766642.2164999992], [-100345.76909999922, 766642.2225000001], {'a': [[-100344.84769999981, 766642.1664000005], [-100345.02722457946, 766646.7405769391], 1, 0, 6.223259474151225, 6.593662940400221, 0.6924846219966052]}, [-100344.84820000082, 766642.1818000004], [-100151.36700000055, 766650.1786000002], [-100158.29839999974, 766305.5562999994], [-100158.29250000045, 766304.9081999995], [-100158.23650000058, 766304.2640000004], [-100158.13069999963, 766303.6285999995], [-100157.97589999996, 766303.0066999998], [-100157.77339999937, 766302.4031000007], [-100157.52470000088, 766301.8224], [-100157.23159999959, 766301.2689999994], [-100156.89650000073, 766300.7473000009], [-100156.52170000039, 766300.2609999999], [-100156.11030000076, 766299.8138999995], [-100155.66530000046, 766299.4094999991], [-100155.19019999914, 766299.0507999994], [-100154.68850000016, 766298.7404999994], [-100154.1640000008, 766298.4810000006], [-100153.62079999

Processed 128000 features
Processed 129000 features
Processed 130000 features


In [ ]:
#REMOVE AFTER RUNNING
#COMBINING THE TWO FIRE FILES

#Reading files in
fire_dist = pd.read_csv('../intermediate_data/fire_distances.csv')
fire_dist_rest = pd.read_csv('../intermediate_data/fire_distances_rest.csv')

#Combining
frames = [fire_dist, fire_dist_rest]
fire_dist_df = pd.concat(frames)